In [ ]:
import geopandas as gpd
import pandas as pd
import os
import getpass
import matplotlib.pyplot as plt
from earthpy.clip import clip_shp

### NOTES:


###### Bring in necessary geojson files and set your projection for all files

In [ ]:
wd = os.getcwd()
wd = wd[:wd.find('notebooks')]

#crs is set for Central Texas; https://epsg.io/6578; 
crs =  {'init' :'epsg:6578'}

#parks = gpd.read_file(wd+"data/coaparks/parkboundaries.geojson")
#parks = parks.to_crs(crs).explode().reset_index()

quartbuff = gpd.read_file(wd+"data/coaparks_buffer/quarterbuff.shp")
quartbuff.crs={'init' :'epsg:6578'}

pop = gpd.read_file(wd+"data/blockgroups_censusdata/popmerge.shp")
pop = pop.to_crs(crs).reset_index()

race = gpd.read_file(wd+"data/blockgroups_censusdata/racemerge.shp")
race = race.to_crs(crs).reset_index()

income = gpd.read_file(wd+"data/blockgroups_censusdata/incomemerge.shp")
income = income.to_crs(crs).reset_index()

##### Check your projections
https://geopandas.org/projections.html

In [ ]:
quartbuff.crs

In [ ]:
pop.crs

In [ ]:
race.crs

###### Preview the files and clean the data

In [ ]:
pop['fullarea_pop'] = pop['geometry'].area
pop.head()

In [ ]:
race['fullarea_race'] = race['geometry'].area
race.head()

In [ ]:
income['fullarea_income'] = income['geometry'].area
income.head()

In [ ]:
quartbuff['fullarea_buff'] = quartbuff['geometry'].area
quartbuff.head()

### Run spatial analysis on the amount of people distributed within a census block group against the quarter mile park buffer area and the race census data

In [ ]:
rp_intersection = gpd.overlay(race, quartbuff, how='intersection') #https://geopandas.org/set_operations.html
rp_intersection['area_intersec'] = rp_intersection['geometry'].area
rp_intersection.to_file(wd+"data/access/rp_intersec_quarterbuff_ACS17.shp")
rp_intersection

In [ ]:
rp_intersection.crs

In [ ]:
rp_intersection.columns

In [ ]:
rp_intersection[['GEOID10','Total_POP','fullarea_race','LOCATION_N','fullarea_buff','area_intersec','geometry']]

In [ ]:
race_clip=rp_intersection.copy().reset_index()

for val in race_clip:

    race_clip['weight'] = race_clip['area_intersec']/ race_clip['fullarea_race']
    
    race_clip['access_pop'] = race_clip['weight'] * race_clip['Total_POP']
    race_clip['access_nonhis'] = race_clip['weight'] * race_clip['Not Hispan']
    race_clip['access_white'] = race_clip['weight'] * race_clip['White; Not']
    race_clip['access_his_lat'] = race_clip['weight'] * race_clip['Hispanic o']

race_clip.head()

In [ ]:
#race_clip.columns

In [ ]:
race_calc = race_clip[['LOCATION_N', 'access_pop', 'access_nonhis','access_white', 'access_his_lat','geometry']]
access_data = race_calc.dissolve(by='LOCATION_N',as_index=False, aggfunc='sum')

##### The population served by parks is nomalized by dividing the population served by the area of the park for which they are being served 

In [ ]:
access_data['fullarea_park'] = access_data['geometry'].area

access_data['Normalized_byArea'] = access_data['access_pop']/access_data['fullarea_park']
access_data['Normalized_nonhis'] = access_data['access_nonhis']/access_data['fullarea_park']
access_data['Normalized_white'] = access_data['access_white']/access_data['fullarea_park']
access_data['Normalized_hislat'] = access_data['access_his_lat']/access_data['fullarea_park']

In [ ]:
access_data.to_file(wd+"data/access/access_data_race_ACS17.shp")
access_data

#### A dataframe is created with the 'acess_data' geoshapefile. This dataframe is cleaned up and the values are convereted into integers. Finally, we export the dataframe into a csv file and a shapefile. 

In [ ]:
df = pd.DataFrame(access_data)

df['Park_Name']=df['LOCATION_N']
df['Total_Pop_Served'] = df['access_pop'].astype(int)
df['Normalized_byArea'] = df['Normalized_byArea'].astype(int)
df['Non_Hispan'] = df['access_nonhis'].astype(int)
df['Normalized_nonhis'] = df['Normalized_nonhis'].astype(int)
df['White'] = df['access_white'].astype(int)
df['Normalized_white'] = df['Normalized_white'].astype(int)
df['Hispan_Latin'] = df['access_his_lat'].astype(int)
df['Normalized_hislat'] = df['Normalized_hislat'].astype(int)

df.head()


Export to Shapefile

In [ ]:
access_race_final = df.copy().drop(columns=['LOCATION_N','access_pop','access_nonhis','access_white','access_his_lat'])
access_race_final = gpd.GeoDataFrame(access_race_final, geometry='geometry')
access_race_final.to_file(wd+"data/access/access_race_final_ACS17.shp")
access_race_final.head()

Export to CSV

In [ ]:
access_racetable = df.drop(columns=['LOCATION_N','access_pop','access_nonhis','access_white','access_his_lat', 'geometry'])
access_racetable.to_csv(wd+"data/access/access_table_race_ACS17.csv")
access_racetable.head()

Descriptive Statistics

In [ ]:
Race_access_stats = access_racetable.describe()
Race_access_stats.to_csv(wd+"data/access/access_stats_race_ACS17.csv")
Race_access_stats

In [ ]:
#access_racetable.hist('');

### Run spatial analysis on the amount of people distributed within a census block group against the quarter mile park buffer area and the income census data

In [ ]:
ip_intersection = gpd.overlay(income, quartbuff, how='intersection') #https://geopandas.org/set_operations.html
ip_intersection['iarea_intersec'] = ip_intersection['geometry'].area
ip_intersection.to_file(wd+"data/access/ip_intersec_quarterbuff_ACS17.shp")
ip_intersection

In [ ]:
ip_intersection.columns

In [ ]:
ip_intersection[['GEOID10','Total_Pop','fullarea_income','LOCATION_N','fullarea_buff','iarea_intersec','geometry']]

In [ ]:
ip_intersection.columns

In [ ]:
income_clip=ip_intersection.copy().reset_index()

for val in income_clip:

    income_clip['weight'] = income_clip['iarea_intersec']/ income_clip['fullarea_income']
    
    income_clip['access_by_income_TotalPop'] = income_clip['weight'] * income_clip['Total_Pop']
    
    income_clip['Less than $25,000'] = (income_clip['weight'] * income_clip['Less than'])\
                                        +(income_clip['weight'] * income_clip['$10,000 to'])\
                                        +(income_clip['weight'] * income_clip['$15,000 to'])\
                                        +(income_clip['weight'] * income_clip['$20,000 to'])
    
    income_clip['$25,000 to $49,999'] = (income_clip['weight'] * income_clip['$25,000 to'])\
                                        +(income_clip['weight'] * income_clip['$30,000 to'])\
                                        +(income_clip['weight'] * income_clip['$35,000 to'])\
                                        +(income_clip['weight'] * income_clip['$40,000 to'])\
                                        +(income_clip['weight'] * income_clip['$45,000 to'])\
    
    income_clip['$50,000 to $74,999'] = (income_clip['weight'] * income_clip['$50,000 to'])\
                                        +(income_clip['weight'] * income_clip['$60,000 to'])\
    
    income_clip['$75,000 to $99,999'] = (income_clip['weight'] * income_clip['$75,000 to'])
    
    income_clip['$100,000 to $149,999'] = (income_clip['weight'] * income_clip['$100,000 t'])\
                                            +(income_clip['weight'] * income_clip['$125,000 t'])\
    
    income_clip['$150,000 or more'] = (income_clip['weight'] * income_clip['$150,000 t'])\
                                        +(income_clip['weight'] * income_clip['$200,000 o'])\
    
income_clip.head()

In [ ]:
income_clip.columns

In [ ]:
income_calc = income_clip[['LOCATION_N', 'access_by_income_TotalPop', 'Less than $25,000','$25,000 to $49,999',\
                           '$50,000 to $74,999','$75,000 to $99,999','$100,000 to $149,999','$150,000 or more','geometry']]

income_access_data = income_calc.dissolve(by='LOCATION_N',as_index=False, aggfunc='sum')

##### The population served by parks is nomalized by dividing the population served by the area of the park for which they are being served 

In [ ]:
income_access_data['fullarea_p'] = income_access_data['geometry'].area

income_access_data['Normalized_byArea'] = income_access_data['access_by_income_TotalPop']/income_access_data['fullarea_p']


In [ ]:
income_access_data.to_file(wd+"data/access/access_data_income_ACS17.shp")
income_access_data

#### A dataframe is created with the 'acess_data' geoshapefile. This dataframe is cleaned up and the values are convereted into integers. Finally, we export the dataframe into a csv file and a shapefile. 

In [ ]:
df = pd.DataFrame(income_access_data)
df['Park_Name']=df['LOCATION_N']

df['Total Pop Served'] = df['access_by_income_TotalPop'].astype(int)
df['Less than $25,000'] = df['Less than $25,000'].astype(int)
df['$25,000 to $49,999'] = df['$25,000 to $49,999'].astype(int)
df['$50,000 to $74,999'] = df['$50,000 to $74,999'].astype(int)
df['$75,000 to $99,999'] = df['$75,000 to $99,999'].astype(int)
df['$100,000 to $149,999'] = df['$100,000 to $149,999'].astype(int)
df['$150,000 or more'] = df['$150,000 or more'].astype(int)







In [ ]:
access_income_final = df.copy().drop(columns=['access_by_income_TotalPop'])
access_income_final = gpd.GeoDataFrame(access_income_final, geometry='geometry')
access_income_final.to_file(wd+"data/access/access_income_final_ACS17.shp")
access_income_final.head()

In [ ]:
access_incometable = df.drop(columns=['access_by_income_TotalPop','geometry'])
access_incometable.to_csv(wd+"data/access/access_table_income_ACS17.csv")
access_incometable.head()

In [ ]:
access_income_stats = access_incometable.describe()
access_income_stats.to_csv(wd+"data/access/access_stats_income_ACS17.csv")
access_income_stats

### END OF CODE

##### The following are cells of unused code or code that does not work in its current form